<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/08_document_classification_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNを使った文書分類
* RNNの出力を文書の潜在表現として利用し、文書分類を行う。
* 単語の埋め込みも含めて学習する。

## 準備
* ランタイムのタイプをGPUにしておこう。


* 再現性の確保については下記を参照。
 * https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

## IMDbデータの準備
* IMDbデータセットは`torchtext.datasets`から使うこともできる。
 * https://pytorch.org/text/stable/datasets.html
 * https://torchtext.readthedocs.io/en/latest/datasets.html
* だが、語彙集合を作成するために使う`torchtext.vocab.build_vocab_from_iterator`という関数がとても遅い・・・。
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* なので、ここではCountVectorizerで語彙集合を作ることにした。

In [ ]:
!pip install ml_datasets

In [ ]:
from ml_datasets import imdb

train_data, test_data = imdb()
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

# ラベルは整数の1と0に変換しておく
label_id = { "pos":1, "neg":0 }

train_labels = [label_id[label] for label in train_labels]
test_labels = [label_id[label] for label in test_labels]

In [ ]:
train_texts[0]

In [ ]:
train_labels[0]

## sklearnのCountVectorizerを使ってトークン化
* 一般に、語彙集合を確定させるときは、訓練データだけを使う。

### CountVectorizerで語彙集合を作成
* ここで作った語彙集合のサイズを、あとで絞る。
* `CountVectorizer`の`token_pattern`を指定し、1文字の単語が消えないようにする。
 * ここでは、テキストを、bag-of-wordsとしてではなく、単語の列としてモデル化する。
 * その場合、例えば冠詞"a"が消えてしまうのは避けたい。
* ただし、下の正規表現だと、punctuationは消えてしまう。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# デフォルトの正規表現とは異なる正規表現を使う
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")

# 訓練データのテキストをトークン化する
X = vectorizer.fit_transform(train_texts)

# 語彙集合を取得する
vocabulary = vectorizer.get_feature_names_out()
print(len(vocabulary))

### 単語のdocument frequencyを計算
* このままだと語彙サイズが大きすぎる。
* 後で、語彙を絞り込むために、document frequencyを使うことにする。
 * document frequencyが小さい単語は、未知語として扱うことにする。

In [ ]:
doc_freq = np.array((X > 0).sum(0)).squeeze()

* document frequencyでトップ10の単語を見てみる。

In [ ]:
print(vocabulary[np.argsort(- doc_freq)][:10])

## `torchtext`の語彙集合の作成
* 作り方は下記のリンク先を参照。
 * https://pytorch.org/text/stable/vocab.html#id1

### 単語をキーとするOrderedDictを作成
 * torchtextで語彙集合を作成するとき、OrderedDictを渡すことになっているため。

In [ ]:
from collections import OrderedDict

vocab_ordered_dict = OrderedDict(zip(vocabulary, doc_freq))

In [ ]:
len(vocab_ordered_dict)

### `torchtext`の語彙集合を作成
* ここでは、document frequencyが10未満の単語を未知語とすることで、語彙サイズを抑えている。
 * ここはチューニングする余地がある。

In [ ]:
from torchtext.vocab import vocab

# 未知語は全て"<unk>"という特殊なトークンへ置き換えることにする
unknown_token = "<unk>"

# padding用のトークンを作っておく
padding_token = "<pad>"

# OrderedDictをもとにtorchtextでの語彙集合を作成
#   min_freqを指定すると、低頻度語は全て未知語として扱われる。
#   ここで、OrderedDictの各keyに対応するvalueが用いられる。
#   （つまり、document frequency以外の値で未知語を決めても構わない。）
vocab = vocab(
    vocab_ordered_dict, min_freq=10,
    specials=[unknown_token, padding_token],
    )

# 語彙にない単語のインデックスは全て"<unk>"と同じインデックスになるよう、設定する
vocab.set_default_index(vocab[unknown_token])

print(len(vocab))

## テキストをインデックスの列へ変換する関数を定義

In [ ]:
# fit済みのCountVectorizerから、前処理とトークナイザを持ってくる
preprocessor = vectorizer.build_preprocessor()
tokenizer = vectorizer.build_tokenizer()

# 前処理、トークナイザ、インデックス列への変換を、一つの処理としてまとめる
text_pipeline = lambda x: vocab(tokenizer(preprocessor(x)))

* このトークナイザでトークン化すると、punctuationは消えることに注意。

In [ ]:
text = "This is a pen."
print(preprocessor(text))
print(tokenizer(preprocessor(text)))
print(text_pipeline(text))

* 特殊なトークンには、以下のインデックスが割り振られている。

In [ ]:
vocab.get_stoi()['<pad>']

In [ ]:
vocab.get_stoi()['<unk>']

* インデックスから単語への変換は、以下のように行うことができる。

In [ ]:
vocab.get_itos()[1001]

## Dataset

### 全テキストのトークン化
* 単語インデックスの列への変換から、テンソルへの変換まで、おこなっている。

In [ ]:
train_tokens = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in train_texts]

In [ ]:
print(train_tokens[0])

In [ ]:
test_tokens = [torch.tensor(text_pipeline(text), dtype=torch.int64) for text in test_texts]

### 自前のデータセットの定義

In [ ]:
class MyTextDataset(Dataset):
  def __init__(self, labels, tokens):
    self.labels = labels
    self.tokens = tokens

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):
    return self.labels[index], self.tokens[index]

train_dataset = MyTextDataset(train_labels, train_tokens)
test_dataset = MyTextDataset(test_labels, test_tokens)

### validation setを切り分ける

In [ ]:
valid_size = len(train_dataset) // 5
train_size = len(train_dataset) - valid_size
test_size = len(test_dataset)

split_train_, split_valid_ = random_split(train_dataset, [train_size, valid_size])

## DataLoader

### RNNの入力として使えるミニバッチを作る関数を定義
* paddingして、同じミニバッチに含まれる単語id列の長さを揃える関数。
* この関数は、DataLoaderクラスのインスタンスを作るときに、`collate_fn`の値として指定する。

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# paddingに使うトークンのインデックスを取得
PAD_IDX = vocab.get_stoi()[padding_token]

def collate_batch(batch):
  labels, tokens = zip(*batch)
  labels = torch.tensor(labels, dtype=torch.int64)
  tokens = pad_sequence(tokens, padding_value=PAD_IDX)
  return labels, tokens

* 試しにバッチサイズ4でDataLoaderを作って、ミニバッチの中身を見てみる。

In [ ]:
BATCH_SIZE = 4
train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [ ]:
labels, tokens = next(iter(train_loader))
print(labels)
print(tokens.shape)
print(tokens)

* ミニバッチのサイズは64にしておく。
 * ここはチューニングできる。

In [ ]:
BATCH_SIZE = 64
train_loader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

## モデルの定義
* LSTMを使う（GRUに変えても良い）
 * http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
VOCAB_SIZE = len(vocab)
NUM_CLASS = 2
EMSIZE = 64
HID_DIM = 64

In [ ]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx, p=0.0):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.dropout = p

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(emb_dim, hid_dim)
    self.fc = nn.Linear(hid_dim * 2, num_class)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # embeddedの形は(トークン列長, バッチサイズ, 埋め込み次元数)
    embedded = self.dropout(self.embedding(src))

    # outputsの形は(トークン列長, バッチサイズ, 隠れ状態の次元数)
    # hiddenの形は(1, バッチサイズ, 隠れ状態の次元数)
    outputs, (hidden, _) = self.rnn(embedded)

    # mean_outputsの形は(バッチサイズ, 隠れ状態の次元数)
    # hiddenの形は(バッチサイズ, 隠れ状態の次元数)
    mean_outputs = outputs.mean(0)
    hidden = hidden.squeeze()

    return self.fc(torch.cat((mean_outputs, hidden), dim=1))

In [ ]:
model = RNNTextSentiment(
    EMSIZE, HID_DIM, NUM_CLASS, VOCAB_SIZE,
    padding_idx=PAD_IDX,
    p=0.5,
    ).to(device)

## 最適化アルゴリズム

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

* パラメータの数を数えてみる。

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters.')

## 損失関数

In [ ]:
criterion = nn.CrossEntropyLoss()

## 訓練を行なう関数

In [ ]:
def train(dataloader, clip=1.):
  model.train()
  total_loss, total_acc, total_count = 0, 0, 0
  for label, text in dataloader:
    label, text = label.to(device), text.to(device)
    output = model(text)
    loss = criterion(output, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()
    n_instances = label.size(0)
    total_loss += loss.item() * n_instances
    total_acc += (output.argmax(1) == label).sum().item()
    total_count += n_instances
  return total_loss / total_count, total_acc / total_count

## 評価を行なう関数

In [ ]:
def evaluate(dataloader):
  model.eval()
  total_loss, total_acc, total_count = 0, 0, 0
  for label, text in dataloader:
    label, text = label.to(device), text.to(device)
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, label)
      n_instances = label.size(0)
      total_loss += loss.item() * n_instances
      total_acc += (output.argmax(1) == label).sum().item()
      total_count += n_instances
  return total_loss / total_count, total_acc / total_count

### 時間表示用の関数

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time // 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## 学習の実行

In [ ]:
N_EPOCHS = 10

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(train_loader)
  valid_loss, valid_acc = evaluate(valid_loader)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc*100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc*100:.1f}% (valid)')

# 課題6
* 上のコードを動かして、感情分析を実践してみよう。
* 余裕があれば、ハイパーパラメータをチューニングして、分類性能を上げてみよう。
 * 例えば、LSTMのレイヤ数を2以上にすると、性能は上がるだろうか？

## テストデータで評価

In [ ]:
loss, acc = evaluate(test_loader)
print(f'\tLoss {loss:.4f} (test)\t|\tAcc {acc*100:.1f}% (test)')